# CAPSTONE PROJECT - PART 1
__Michael Gat__  
__General Assembly Santa Monica, Data Science Immersive, Summer 2016__

In this notebook, we'll work through the data, cleaning it up and turning it into classification variables as we go. In some cases this is rather simple. In others, we may need to explore the data more thoroughly in order to identify the values that need to become columns in the complete dataset. The objective is a dataset including only numeric columns that can be used for classification purposes.

In [144]:
# IMPORT LIBRARIES ############################################################

import pandas as pd
import numpy as np

## DATA WRANGLING FUNCTIONS
These functions are used below for cleaning data and assigning numeric codes in place of text categories.

In [145]:
def age_group_classify(age):
    #
    # Assign simple numeric group for age based on age group reported
    #
    if age == '[0-10)':
        return 0
    elif age == '[10-20)':
        return 1
    elif age == '[20-30)':
        return 2
    elif age == '[30-40)':
        return 3
    elif age == '[40-50)':
        return 4
    elif age == '[50-60)':
        return 5
    elif age == '[60-70)':
        return 6
    elif age == '[70-80)':
        return 7
    elif age == '[80-90)':
        return 8
    elif age == '[90-100)':
        return 9
    else:
        # Going to just go on the assumption that > 100 is a small enough group to be essentially an outlier
        # I have confirmed that in current medical practice this is the case and I won't bother to break it
        # down further, particularly given the tiny sample size.
        return 10

In [146]:
def race_classify(race):
    #
    # Assign simple numeric group for race
    #
    if race == 'Caucasian':
        return 1
    elif race == 'AfricanAmerican':
        return 2
    elif race == 'Hispanic':
        return 3
    elif race == 'Asian':
        return 4
    else:
        # All others are categorized either as 'Other' or '?' in the original data
        # For the purposes of classification, they are treated as a single group
        return 0

In [147]:
def readmit_group_classify(readmit):
    #
    # Simple target for readmit status, 0 or 1. May make more complex later
    # 0 - not readmitted
    # 0 - readmitted > 30 days
    # 1 - readmitted < 30 days
    #
    if readmit == 'NO':
        return 0
    elif readmit == '>30':
        return 0
    else:
        return 1

In [148]:
def numeric_classification(column_val):
    #
    # Simply reclassify all yes/no (or "change/no") variables as numerics
    #
    if column_val == 'No':
        return 0
    else:
        return 1

## READ IN DATA
Read dataset and immediately drop fields that are mostly unpopulated.

In [149]:
df = pd.read_csv('diabetic_data.csv')

# Drop unnecessary columns

# Payer code and weight are mostly unreported and thus cannot provide useful data
del df['payer_code']
del df['weight']


In [150]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,?,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,?,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,?,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,?,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


## CLEAN/CLASSIFY THE DATA
### Simple variables
Age, readmit status (the target) and all the yes/no fields (mostly medications administered) are relatively simple. functions to help do this were included above.

In [151]:
# Assign numeric age groups then drop age column
df['age_group'] = df['age'].apply(age_group_classify)
del df['age']

# Assign numeric readmit classifier then drop readmitted column
df['readmit'] = df['readmitted'].apply(readmit_group_classify)
del df['readmitted']

# Assign numeric value for yes/no and a few other situations
values_map = {'No': 0, 'Yes': 1, 'Ch': 1, 'Down': 1, 'Steady': 2, 'Up': 3, 'Female': 0, 'Male': 1, 'Unknown/Invalid': 0}
df = df.replace(values_map)

df.gender.value_counts()

0    54711
1    47055
Name: gender, dtype: int64

### Build classifier for race

In [152]:
# Got to do something about race. Not always politically correct, but need to
# But first what are they?

df.race.value_counts()

Caucasian          76099
AfricanAmerican    19210
?                   2273
Hispanic            2037
Other               1506
Asian                641
Name: race, dtype: int64

In [153]:
# Now replace race with a numeric as well.
# A bit out of sequence, I wrote the above function after the first bit of analysis

df.race = df.race.apply(race_classify)

### Build classifier for max glucose
Deal with max_glu_serum (usually blank)
Most patients will not have a fasting glucose test, but where it happens and the result is elevated, it may be of interest. For simplicity, going to classify those situations where it is measured and elevated as a 1, all other conditions as 0.

_(__Note to self__: The fact that the MD wanted it done in the first place may be a relevant indicator regardless of result, as it suggests the the MD saw something in the patient's history or condition that is not captured in this dataset. But as the purpose here is to evaluate known facts not the MD's thought process, we'll only look at the serum level when it is measured. However, this is an interesting thing to look at in the future if the data can be gathered.)_

In [154]:
# Get a look at the values in that field

df.max_glu_serum.value_counts()

None    96420
Norm     2597
>200     1485
>300     1264
Name: max_glu_serum, dtype: int64

In [155]:
# Going to use "none" and "normal" as the zero bucket. Going to simplify at this point and make it 1
# if there is a significantly elevated level (>200) and ignore for now the distinction between
# > 200 and > 300 Possibly come back to this

df.max_glu_serum = [1 if ((row == '>200') | (row == '>300')) else 0 for row in df.max_glu_serum]
df.max_glu_serum.value_counts()

0    99017
1     2749
Name: max_glu_serum, dtype: int64

### Classifier for A1C
This one is similar in most ways to the previous one and will be handled similarly. If it's measured and elevated, it will be classified as a 1, otherwise 0.

In [156]:
# Same for A1C.
# Where it exists and is elevated, it matters, otherwise it does not.

df.A1Cresult.value_counts()

None    84748
>8       8216
Norm     4990
>7       3812
Name: A1Cresult, dtype: int64

In [157]:
df.A1Cresult = [1 if ((row == '>7') | (row == '>8')) else 0 for row in df.A1Cresult]
df.A1Cresult.value_counts()

0    89738
1    12028
Name: A1Cresult, dtype: int64

### Medical Specialty
Have a look at this one. Per the documentation of the dataset, it's mostly empty and appears to be fairly inconsistent.

In [158]:
# Examine medical specialty, it seems to be mostly blank
df.medical_specialty.value_counts()

?                                    49949
InternalMedicine                     14635
Emergency/Trauma                      7565
Family/GeneralPractice                7440
Cardiology                            5352
Surgery-General                       3099
Nephrology                            1613
Orthopedics                           1400
Orthopedics-Reconstructive            1233
Radiologist                           1140
Pulmonology                            871
Psychiatry                             854
Urology                                685
ObstetricsandGynecology                671
Surgery-Cardiovascular/Thoracic        652
Gastroenterology                       564
Surgery-Vascular                       533
Surgery-Neuro                          468
PhysicalMedicineandRehabilitation      391
Oncology                               348
Pediatrics                             254
Hematology/Oncology                    207
Neurology                              203
Pediatrics-

In [159]:
df['medical_specialty'].nunique()

73

### Drop medical specialty column
Decision is to drop medical specialty as a column. While this seems like it might be useful information, the reality is that we don't have it half the time. Where it is available, it is completely unstructured and may reflect differences in reporting this information by different hospitals. The vast bulk of the categories available account for less than 1% of cases. In some cases multiple specialities are combined and done so in different ways. It is not clear from the data whether this information reflects multiple physicians, multiple specialties or some other cases. Given this, it would be unreasonable to draw conclusions from this information.

This data might be more useful if a clinical specialist could provide insight into how to better lump them into broad categories, but such an effort is beyond the scope of this project.

In [160]:
del df['medical_specialty']
df.head()

,encounter_id,patient_nbr,race,gender,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,age_group,readmit
0,2278392,8222157,1,0,6,25,1,1,41,0,...,0,0,0,0,0,0,0,0,0,0
1,149190,55629189,1,0,1,1,7,3,59,0,...,3,0,0,0,0,0,1,1,1,0
2,64410,86047875,2,0,1,1,7,2,11,5,...,0,0,0,0,0,0,0,1,2,0
3,500364,82442376,1,1,1,1,7,2,44,1,...,3,0,0,0,0,0,1,1,3,0
4,16680,42519267,1,1,1,1,7,1,51,0,...,2,0,0,0,0,0,1,1,4,0


## DIAGNOSTIC CODE EXPLORATION / CLASSIFICATION

Diagnostic codes are provided in three columns, each containing a diagnostic code. A patient may have only one, or may have all three. In fact, there may be more associated with the original patient record that are not available to us.

Given the large number of diagnostic codes (882), most of which are used for only a single patient, it makes little sense to look at all of them. We'll focus the attention (for now anyway) on the most common ones, that are used in at least 3% of all records.

### Explore the diagnostic code data in all three fields

Get an idea of how common some of these codes are, whether they exist in more than one column, etc.

In [161]:
print df.diag_1.nunique()
df.diag_1.value_counts()

717


428       6862
414       6581
786       4016
410       3614
486       3508
427       2766
491       2275
715       2151
682       2042
434       2028
780       2019
996       1967
276       1889
38        1688
250.8     1680
599       1595
584       1520
V57       1207
250.6     1183
518       1115
820       1082
577       1057
493       1056
435       1016
562        989
574        965
296        896
560        876
250.7      871
250.13     851
          ... 
84           1
57           1
791          1
832          1
318          1
365          1
523          1
649          1
133          1
363          1
160          1
114          1
957          1
955          1
885          1
10           1
804          1
E909         1
837          1
V07          1
412          1
870          1
839          1
V70          1
637          1
838          1
982          1
834          1
817          1
314          1
Name: diag_1, dtype: int64

In [162]:
print df.diag_2.nunique()
df.diag_2.value_counts()

749


276       6752
428       6662
250       6071
427       5036
401       3736
496       3305
599       3288
403       2823
414       2650
411       2566
250.02    2074
707       1999
585       1871
584       1649
491       1545
250.01    1523
285       1520
780       1491
425       1434
682       1433
486       1379
518       1355
424       1071
413       1042
250.6      895
493        881
305        702
786        644
280        606
998        571
          ... 
353          1
350          1
140          1
145          1
V60          1
232          1
235          1
V69          1
325          1
523          1
927          1
832          1
843          1
948          1
947          1
846          1
944          1
912          1
917          1
915          1
674          1
670          1
212          1
942          1
580          1
E890         1
E929         1
529          1
833          1
270          1
Name: diag_2, dtype: int64

In [163]:
print df.diag_3.nunique()
df.diag_3.value_counts()

790


250       11555
401        8289
276        5175
428        4577
427        3955
414        3664
496        2605
403        2357
585        1992
272        1969
599        1941
?          1423
V45        1389
250.02     1369
707        1360
780        1334
285        1200
425        1136
250.6      1080
424        1063
584         963
305         924
250.01      915
682         887
518         854
41          727
493         694
278         680
530         625
786         584
          ...  
370           1
E876          1
755           1
863           1
542           1
E949          1
540           1
186           1
226           1
970           1
971           1
972           1
E922          1
361           1
27            1
57            1
538           1
853           1
158           1
811           1
671           1
524           1
193           1
195           1
674           1
E955          1
893           1
E892          1
236           1
E886          1
Name: diag_3, dtype: int

### Get a better look at the diagnostic codes, sum them up and get the most common ones.

We have three diagnostic codes in three columns. It is unknown which is the most important. However, all patients have at least one. Some have three. A missing bit of data is additonal IDC codes entered in the patient's EMR that may be more important than the ones included here. As we can only speculate about what might exist in an EMR that we have no access to, we will work with the data we have.

In [164]:
# Get count of the codes in the first diagnostic field and make them into a dataframe
df_diag_count1 = df[['encounter_id', 'diag_1']].groupby('diag_1').count()
df_diag_count1.columns = ['count1']
df_diag_count1.index.name = ['IDC']

# Get count of the codes in the second diagnostic field and make them into a dataframe
df_diag_count2 = df[['encounter_id', 'diag_2']].groupby('diag_2').count()
df_diag_count2.columns = ['count2']
df_diag_count2.index.name = ['IDC']

# Get count of the codes in the third diagnostic field and make them into a dataframe
df_diag_count3 = df[['encounter_id', 'diag_3']].groupby('diag_3').count()
df_diag_count3.columns = ['count3']
df_diag_count3.index.name = ['IDC']

In [165]:
# Now take the counts of each IDC code from each of the columns and sum them up to yield a single
# column with the total count for each diagnostic code
# Start by joining the three columns on the IDC code which is the index
# This will result in a single dataframe with three columns.

df_temp = df_diag_count1.join(df_diag_count2, how = 'outer')
df_IDC_counts = df_temp.join(df_diag_count3, how = 'outer')
df_IDC_counts.fillna(value=0, inplace=True)

# Sum the three columns into a single totals column
# Drop the three columns and then sort on the total

df_IDC_counts['total'] = df_IDC_counts['count1'] + df_IDC_counts['count2'] + df_IDC_counts['count3']
df_IDC_counts.drop(df_IDC_counts.columns[[0, 1, 2]], axis=1, inplace=True)
df_IDC_counts.index.name = 'IDC'
df_IDC_counts.sort_values('total', inplace=True, ascending=False)

# All encounters in this dataset have a diabetic condition, whether it appears in the three diagnostic
# codes or not. Thus, a diabetes diagnosis (250) provides no useful information for our study. We will drop
# those codes from our list of "interesting" IDC codes.

df_IDC_counts = df_IDC_counts[~df_IDC_counts.index.str.contains('250')]
print 'Total number of IDC codes used in dataset: ' + str(len(df_IDC_counts.index))
print \

df_IDC_counts.head()

Total number of IDC codes used in dataset: 882



,total
IDC,
428,18101.0
276,13816.0
414,12895.0
401,12371.0
427,11757.0


In [166]:
# Going to select the most common 20 diagnostic codes.
# As it just so happens, this is almost exactly the number of codes that appear in at least 3% of the records
# If this works, I may expand the analysis, but for now, looking at only the factors that apply to at least 3%
# of the sample seems reasonable in looking for the dominant features and may even be overkill.
#
IDC_list = df_IDC_counts.index.tolist()[:20]
IDC_list

['428',
 '276',
 '414',
 '401',
 '427',
 '599',
 '496',
 '403',
 '486',
 '786',
 '780',
 '491',
 '410',
 '682',
 '584',
 '585',
 '707',
 '518',
 '411',
 '285']

### No great surprises in the diagnoses we came up with
For the most part, these are conditions that are consistent with or related to a diabetic condition, with various forms of heart disease, dehydration/fluid issues, and hypertension topping the list.

A full description of the diagnostic codes is available at: http://www.icd9data.com/2015/Volume1/default.htm

In [167]:
# Now, create columns for each of these codes in the main dataframe
# then drop the original columns containing diagnostic codes

for i in IDC_list:
    df[i] = 0
    df[i] = np.where(((df['diag_1'] == i) | \
                      (df['diag_2'] == i) | \
                      (df['diag_3'] == i)), 1, 0)
    
df.drop('diag_1', axis=1, inplace=True)
df.drop('diag_2', axis=1, inplace=True)
df.drop('diag_3', axis=1, inplace=True)

In [168]:
# Finally, now that everything is really clean, drop the encounter ID and patient #
# Neither is going to be useful going forward.

df.drop('encounter_id', axis=1, inplace=True)
df.drop('patient_nbr', axis=1, inplace=True)
df.head()

,race,gender,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,780,491,410,682,584,585,707,518,411,285
0,1,0,6,25,1,1,41,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,7,3,59,0,18,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,1,1,7,2,11,5,13,2,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,7,2,44,1,16,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,7,1,51,0,8,0,...,0,0,0,0,0,0,0,0,0,0


In [169]:
# Still seem to have some NaNs somewhere, so fix it before we write it out
df.fillna(value=0, inplace=True)
df.to_csv('diabetic_data_clean.csv', index=False)